In [1]:
from chessrl.cnnextractor import CustomCNNExtractor
import chess_gymnasium_env

from chessrl.utils import action_mask_fn

In [2]:
import os
import numpy as np

In [3]:
import gymnasium as gym

In [4]:
from sb3_contrib import MaskablePPO
from sb3_contrib.common.envs import InvalidActionEnvDiscrete
from sb3_contrib.common.maskable.evaluation import evaluate_policy
from sb3_contrib.common.maskable.utils import get_action_masks
# This is a drop-in replacement for EvalCallback
from sb3_contrib.common.wrappers import ActionMasker
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env

In [5]:
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv

In [ ]:
model = MaskablePPO.load("data/ppo_mask")

In [7]:
render_env = gym.make('chess_gymnasium_env/ChessEnv-v0', render_mode="human")
render_env = ActionMasker(render_env, action_mask_fn)

In [8]:
obs, _ = render_env.reset()
while True:
    # Retrieve current action mask
    action_masks = get_action_masks(render_env)
    action, _states = model.predict(obs, action_masks=action_masks)
    obs, reward, terminated, truncated, info = render_env.step(action)

    if terminated or truncated:
        break

In [9]:
render_env.close()